## Step 1: Install Dependencies

In [ ]:
!pip install -q langchain-core>=0.3.56 langchain-community>=0.3.12 langgraph>=0.3.25
!pip install -q langchain-mcp-adapters==0.0.9 mcp>=1.6.0
!pip install -q arize-phoenix>=8.28.1
!pip install -q opentelemetry-api>=1.32.1 opentelemetry-sdk>=1.32.1 opentelemetry-exporter-otlp>=1.32.1
!pip install -q oci>=2.154.0 oracledb>=3.1.1
!pip install -q rapidfuzz>=3.13.0 sentence-transformers>=2.2.2 numpy>=1.24.0
!pip install -q gradio>=5.0.0 pyngrok

print("✅ All dependencies installed successfully!")

## Step 2: Upload Configuration Files

Upload the following files from your local machine:
1. `config.properties` - Your OCI and DB configuration
2. `oci_api_key.pem` - Your OCI API key file
3. Wallet ZIP - Your Oracle ATP wallet (e.g., `Wallet_ItsmyATPVectorDBs.zip`)

In [ ]:
from google.colab import files
import os
import zipfile
from pathlib import Path

print("📤 Upload config.properties...")
uploaded = files.upload()
if 'config.properties' in uploaded:
    print("✅ config.properties uploaded")
else:
    print("❌ config.properties not found. Please upload it.")

print("\n📤 Upload oci_api_key.pem...")
uploaded = files.upload()
if 'oci_api_key.pem' in uploaded:
    print("✅ oci_api_key.pem uploaded")
else:
    print("❌ oci_api_key.pem not found. Please upload it.")

print("\n📤 Upload Oracle wallet ZIP file...")
uploaded = files.upload()
wallet_file = list(uploaded.keys())[0] if uploaded else None

if wallet_file and wallet_file.endswith('.zip'):
    # Extract wallet
    wallet_dir = 'Wallet_ItsmyATPVectorDBs'
    os.makedirs(wallet_dir, exist_ok=True)
    with zipfile.ZipFile(wallet_file, 'r') as zip_ref:
        zip_ref.extractall(wallet_dir)
    print(f"✅ Wallet extracted to {wallet_dir}/")
    os.remove(wallet_file)  # Clean up zip
else:
    print("❌ Wallet ZIP not found. Please upload it.")

## Step 3: Create config_loader.py

In [ ]:
%%writefile config_loader.py
"""
Shared configuration loader for OCI GenAI Bot.
Reads config.properties, writes OCI SDK config, and applies DB env variables.
"""
import os
import re
from pathlib import Path
from typing import Dict
import logging

logger = logging.getLogger(__name__)


def load_properties(file_path: str = None) -> Dict[str, str]:
    file_path = file_path or str(Path(os.getcwd()) / 'config.properties')
    cfg: Dict[str, str] = {}
    if not os.path.exists(file_path):
        logger.warning("config.properties not found at %s", file_path)
        return cfg
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            if '=' in line:
                k, v = line.split('=', 1)
                k = k.strip()
                v = v.strip()
                if (v.startswith('"') and v.endswith('"')) or (v.startswith("'") and v.endswith("'")):
                    v = v[1:-1]
                cfg[k] = v
    return cfg


def ensure_oci_config(props: Dict[str, str]) -> None:
    user = props.get('OCI_USER')
    fingerprint = props.get('OCI_FINGERPRINT')
    key_file = props.get('OCI_KEY_FILE')
    tenancy = props.get('OCI_TENANCY_OCID') or props.get('OCI_COMPARTMENT_OCID')
    endpoint = props.get('OCI_GENAI_ENDPOINT', '')

    if not (user and fingerprint and key_file and tenancy):
        logger.warning("Missing OCI credentials in config.properties (user/fingerprint/key_file/tenancy). Using existing OCI setup if available.")
        return

    m = re.search(r"generativeai\.(.*?)\.oci\.oraclecloud\.com", endpoint)
    region = m.group(1) if m else os.getenv('OCI_REGION', 'us-chicago-1')

    root = Path(os.getcwd())
    key_path = str((root / key_file).resolve())
    oci_dir = root / '.oci'
    oci_dir.mkdir(exist_ok=True)
    config_path = oci_dir / 'config'
    content = (
        "[DEFAULT]\n"
        f"user={user}\n"
        f"fingerprint={fingerprint}\n"
        f"key_file={key_path}\n"
        f"tenancy={tenancy}\n"
        f"region={region}\n"
    )
    config_path.write_text(content, encoding='utf-8')
    os.environ['OCI_CONFIG_FILE'] = str(config_path)
    os.environ['OCI_CLI_PROFILE'] = 'DEFAULT'
    logger.info("OCI SDK config written to %s (region=%s)", config_path, region)


def apply_db_env(props: Dict[str, str]) -> None:
    wallet = props.get('WALLET_PATH')
    if wallet:
        wallet_path = str((Path(os.getcwd()) / wallet).resolve())
        os.environ['TNS_ADMIN'] = wallet_path
        logger.info("TNS_ADMIN set to %s", wallet_path)
    if props.get('USERNAME'):
        os.environ['DB_USERNAME'] = props['USERNAME']
    if props.get('PASSWORD'):
        os.environ['DB_PASSWORD'] = props['PASSWORD']
    if props.get('DB_ALIAS'):
        os.environ['DB_ALIAS'] = props['DB_ALIAS']


def get_oci_llm_params(props: Dict[str, str]) -> Dict[str, str]:
    return {
        'model_id': props.get('OCI_GENAI_MODEL_ID', 'cohere.embed-multilingual-v3.0'),
        'service_endpoint': props.get('OCI_GENAI_ENDPOINT', 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com'),
        'compartment_id': props.get('OCI_COMPARTMENT_OCID'),
    }

## Step 4: Create product_search.py

In [ ]:
%%writefile product_search.py
import os
import sys
import logging
from pathlib import Path
from datetime import datetime
import oracledb
import numpy as np
import difflib
from rapidfuzz import fuzz
from langchain_community.embeddings import OCIGenAIEmbeddings
from config_loader import load_properties, ensure_oci_config, apply_db_env, get_oci_llm_params

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(process)d - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class SearchSimilarProduct:
    def __init__(
            self,
            top_k=5,
            minimal_distance=1.0,
            model_id=None,
            service_endpoint=None,
            compartment_id=None,
            auth_profile="DEFAULT",
            wallet_path=None,
            db_alias=None,
            username=None,
            password=None
    ):
        logger.info("Initializing SearchSimilarProduct...")

        PROPS = load_properties(str(Path(os.getcwd())/ 'config.properties'))
        ensure_oci_config(PROPS)
        apply_db_env(PROPS)

        if wallet_path is None:
            wallet_path = str((Path(os.getcwd()) / PROPS.get('WALLET_PATH', 'Wallet_ItsmyATPVectorDBs')).resolve())
        if db_alias is None:
            db_alias = PROPS.get('DB_ALIAS', 'itsmyatpvectordb_high')
        if username is None:
            username = PROPS.get('USERNAME', 'ADMIN')
        if password is None:
            password = PROPS.get('PASSWORD', '')

        os.environ["TNS_ADMIN"] = wallet_path
        try:
            self.conn = oracledb.connect(
                user=username,
                password=password,
                dsn=db_alias,
                config_dir=wallet_path,
                wallet_location=wallet_path,
                wallet_password=password
            )
            logger.info("✅ Database connection established")
        except Exception as e:
            logger.error(f"❌ Failed to connect to database: {e}", exc_info=True)
            raise
            
        self.top_k = top_k
        self.minimal_distance = minimal_distance

        try:
            if model_id is None or service_endpoint is None or compartment_id is None:
                llm = get_oci_llm_params(PROPS)
                model_id = PROPS.get('OCI_GENAI_EMBEDDING_MODEL_ID', 'cohere.embed-multilingual-v3.0')
                service_endpoint = llm['service_endpoint']
                compartment_id = llm['compartment_id']
            self.embedding = OCIGenAIEmbeddings(
                model_id=model_id,
                service_endpoint=service_endpoint,
                compartment_id=compartment_id,
                auth_profile=auth_profile
            )
            logger.info(f"✅ OCI GenAI Embeddings initialized (model: {model_id})")
        except Exception as e:
            logger.error(f"❌ Failed to initialize embeddings: {e}", exc_info=True)
            raise

        logger.info("📦 Loading Oracle Vectors...")
        self._load_embeddings()

    def _load_embeddings(self):
        try:
            cursor = self.conn.cursor()
            cursor.execute("SELECT id, code, description, vector FROM embeddings_products")
            self.vectors = []
            self.products = []
            for row in cursor.fetchall():
                id_, code, description, blob = row
                vector = np.frombuffer(blob.read(), dtype=np.float32)
                self.vectors.append(vector)
                self.products.append({
                    "id": id_,
                    "code": code,
                    "description": description
                })
            self.vectors = np.array(self.vectors)
            logger.info(f"✅ Loaded {len(self.products)} product embeddings")
        except Exception as e:
            logger.error(f"❌ Failed to load embeddings: {e}", exc_info=True)
            raise

    def _correct_input(self, input_user):
        descriptions = [p["description"] for p in self.products]
        suggestions = difflib.get_close_matches(input_user, descriptions, n=1, cutoff=0.6)
        return suggestions[0] if suggestions else input_user

    def search_similar_products(self, description_input):
        logger.info(f"Searching for products similar to: {description_input[:50]}...")
        description_input = description_input.strip()
        description_corrected = self._correct_input(description_input)

        if description_corrected != description_input:
            logger.info(f"Input corrected from '{description_input}' to '{description_corrected}'")

        results = {
            "consult_original": description_input,
            "consult_used": description_corrected,
            "semantics": [],
            "fallback_fuzzy": []
        }

        try:
            consult_emb = self.embedding.embed_query(description_corrected)
            consult_emb = np.array(consult_emb)

            dists = np.linalg.norm(self.vectors - consult_emb, axis=1)
            top_indices = np.argsort(dists)[:self.top_k]

            for idx in top_indices:
                dist = dists[idx]
                if dist < self.minimal_distance:
                    match = self.products[idx]
                    similarity = 1 / (1 + dist)
                    results["semantics"].append({
                        "id": match["id"],
                        "code": match["code"],
                        "description": match["description"],
                        "similarity": round(similarity * 100, 2),
                        "distance": round(dist, 4)
                    })

            logger.info(f"Found {len(results['semantics'])} semantic matches")

            if not results["semantics"]:
                logger.info("No semantic matches found, falling back to fuzzy matching")
                better_fuzz = []
                for product in self.products:
                    score = fuzz.token_sort_ratio(description_corrected, product["description"])
                    better_fuzz.append((product, score))
                better_fuzz.sort(key=lambda x: x[1], reverse=True)

                for product, score in better_fuzz[:self.top_k]:
                    results["fallback_fuzzy"].append({
                        "id": product["id"],
                        "code": product["code"],
                        "description": product["description"],
                        "score_fuzzy": round(score, 2)
                    })
                
                logger.info(f"Found {len(results['fallback_fuzzy'])} fuzzy matches")

        except Exception as e:
            logger.error(f"Error during product search: {e}", exc_info=True)

        return results

## Step 5: Create server_invoice_items.py (MCP Server)

In [ ]:
%%writefile server_invoice_items.py
import os
import logging
from pathlib import Path
from datetime import datetime
import oracledb
from mcp.server.fastmcp import FastMCP
from product_search import SearchSimilarProduct

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(process)d - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

from config_loader import load_properties, apply_db_env

logger.info("Initializing Invoice Items MCP Server...")

PROPS = load_properties(str(Path(os.getcwd())/ 'config.properties'))
apply_db_env(PROPS)

searcher = SearchSimilarProduct()
logger.info("✅ SearchSimilarProduct initialized")

mcp = FastMCP("InvoiceItemResolver")

WALLET_PATH = os.environ.get("TNS_ADMIN") or PROPS.get("WALLET_PATH", "Wallet_ItsmyATPVectorDBs")
DB_ALIAS = os.environ.get("DB_ALIAS") or PROPS.get("DB_ALIAS", "itsmyatpvectordb_high")
USERNAME = os.environ.get("DB_USERNAME") or PROPS.get("USERNAME", "ADMIN")
PASSWORD = os.environ.get("DB_PASSWORD") or PROPS.get("PASSWORD", "")
if WALLET_PATH:
    os.environ["TNS_ADMIN"] = WALLET_PATH


def execute_query(query: str, params: dict = {}):
    try:
        logger.debug(f"Executing query with params: {params}")
        connection = oracledb.connect(
            user=USERNAME,
            password=PASSWORD,
            dsn=DB_ALIAS,
            config_dir=WALLET_PATH,
            wallet_location=WALLET_PATH,
            wallet_password=PASSWORD
        )
        cursor = connection.cursor()
        cursor.execute(query, params)
        results = cursor.fetchall()
        cursor.close()
        connection.close()
        logger.debug(f"Query returned {len(results)} results")
        return results
    except Exception as e:
        logger.error(f"Database query error: {e}", exc_info=True)
        print(f"[ERROR]: {e}")
        return []

def execute_ean_search(search_terms):
    results = []
    try:
        logger.debug(f"Executing EAN search for: {search_terms}")
        connection = oracledb.connect(
            user=USERNAME,
            password=PASSWORD,
            dsn=DB_ALIAS,
            config_dir=WALLET_PATH,
            wallet_location=WALLET_PATH,
            wallet_password=PASSWORD
        )
        cursor = connection.cursor()
        query = """SELECT * FROM TABLE(fn_advanced_search(:1)) ORDER BY similarity DESC"""
        cursor.execute(query, [search_terms])
        for row in cursor:
            results.append({"code": row[0], "description": row[1], "similarity": row[2]})
        cursor.close()
        connection.close()
        logger.debug(f"EAN search returned {len(results)} results")
    except Exception as e:
        logger.error(f"EAN search error: {e}", exc_info=True)
        return {"error": str(e)}, 500
    return results

@mcp.tool()
def search_vectorized_product(description: str) -> dict:
    """Searches for a product by description using embeddings."""
    logger.info(f"MCP Tool: search_vectorized_product called with: {description[:50]}...")
    result = searcher.search_similar_products(description)
    logger.info(f"search_vectorized_product returned {len(result.get('semantics', []))} semantic results")
    return result

@mcp.tool()
def resolve_ean(description: str) -> dict:
    """Resolves the product's EAN code based on its description."""
    logger.info(f"MCP Tool: resolve_ean called with: {description[:50]}...")
    result = execute_ean_search(description)
    if isinstance(result, list) and result:
        logger.info(f"resolve_ean found EAN: {result[0]['code']}")
        return {"code": result[0]["code"], "description": result[0]["description"], "similarity": result[0]["similarity"]}
    else:
        logger.warning("resolve_ean: No EAN found")
        return {"error": "Search not found by EAN."}

@mcp.tool()
def search_invoices_by_criteria(customer: str = None, state: str = None, price: float = None, ean: str = None, margin: float = 0.05) -> list:
    """Searches for outbound invoices based on customer, state, EAN, and approximate price."""
    logger.info(f"MCP Tool: search_invoices_by_criteria called - customer={customer}, state={state}, ean={ean}, price={price}")
    query = """
        SELECT nf.no_invoice, nf.name_customer, nf.state, nf.date_print,
               inf.no_item, inf.code_ean, inf.description_product, inf.value_unitary
        FROM invoice nf
        JOIN item_invoice inf ON nf.no_invoice = inf.no_invoice
        WHERE 1=1
    """
    params = {}
    query += " AND LOWER(nf.name_customer) LIKE LOWER(:customer)"
    params["customer"] = f"%{customer}%"
    query += " AND LOWER(nf.state) = LOWER(:state)"
    params["state"] = state
    query += " AND inf.code_ean = :ean"
    params["ean"] = ean
    if price is not None:
        query += " AND inf.value_unitary BETWEEN :price_min AND :price_max"
        params["price_min"] = price * (1 - margin)
        params["price_max"] = price * (1 + margin)
    result = execute_query(query, params)
    logger.info(f"search_invoices_by_criteria found {len(result)} invoices")
    return [
        dict(zip(["no_invoice", "name_customer", "state", "date_print", "no_item", "code_ean", "description_product", "value_unitary"], row))
        for row in result
    ]

if __name__ == "__main__":
    logger.info("="*60)
    logger.info("Starting MCP Server - InvoiceItemResolver")
    logger.info("="*60)
    mcp.run(transport="stdio")

## Step 6: Create main.py (Agent)

In [ ]:
%%writefile main.py
import asyncio
import logging
import os
import sys
from datetime import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient

import phoenix as px
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(process)d - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

from config_loader import load_properties, ensure_oci_config, apply_db_env, get_oci_llm_params

PROPS = load_properties(os.path.join(os.getcwd(), 'config.properties'))
ensure_oci_config(PROPS)
apply_db_env(PROPS)

logger.info("Starting Phoenix observability platform...")
px.launch_app()
logger.info("✅ Phoenix launched on http://localhost:6006")

resource = Resource(attributes={"service.name": "ollama_oraclegenai_trace"})
provider = TracerProvider(resource=resource)
trace.set_tracer_provider(provider)
otlp_exporter = OTLPSpanExporter(endpoint="http://localhost:6006/v1/traces")
span_processor = BatchSpanProcessor(otlp_exporter)
provider.add_span_processor(span_processor)
tracer = trace.get_tracer(__name__)

class MemoryState:
    def __init__(self):
        self.messages = []

_llm_params = get_oci_llm_params(PROPS)
llm = ChatOCIGenAI(
    model_id=_llm_params['model_id'],
    service_endpoint=_llm_params['service_endpoint'],
    compartment_id=_llm_params['compartment_id'],
    auth_profile="DEFAULT",
    model_kwargs={"temperature": 0.1, "top_p": 0.75, "max_tokens": 2000}
)

prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are an agent responsible for resolving inconsistencies in customer return invoices.
        Your goal is to find the original outbound invoice issued by the company.
        
        1. Use search_vectorized_product to find the most likely EAN from product description.
        2. Use resolve_ean to obtain the most probable EAN.
        3. Use search_invoices_by_criteria with EAN, customer, price, and location.
        
        If found, return: invoice number, customer, state, EAN, description, price.
        If not found: "EAN not found with the provided criteria."
    """),
    ("placeholder", "{messages}")
])

async def main():
    logger.info("="*60)
    logger.info("Initializing OCI GenAI Bot MCP Client")
    logger.info("="*60)
    
    async with MultiServerMCPClient({
        "InvoiceItemResolver": {
            "command": "python",
            "args": ["-u", "server_invoice_items.py"],
            "transport": "stdio",
        }
    }) as client:
        tools = client.get_tools()
        if not tools:
            logger.error("❌ No MCP tools loaded")
            return

        logger.info(f"🛠️ Loaded tools: {[t.name for t in tools]}")
        print("🛠️ Loaded tools:", [t.name for t in tools])

        memory_state = MemoryState()
        agent_executor = create_react_agent(model=llm, tools=tools, prompt=prompt)

        logger.info("🤖 Agent ready to process queries")
        print("🤖 READY")
        
        while True:
            query = input("You: ")
            if query.lower() in ["quit", "exit"]:
                break
            if not query.strip():
                continue

            memory_state.messages.append(HumanMessage(content=query))
            try:
                result = await agent_executor.ainvoke({"messages": memory_state.messages})
                new_messages = result.get("messages", [])
                memory_state.messages = []
                response_content = new_messages[-1].content
                
                if isinstance(response_content, list):
                    response_text = "".join([part.get("text", "") if isinstance(part, dict) else str(part) for part in response_content])
                else:
                    response_text = str(response_content)

                print("ASSIST_BEGIN")
                print(response_text)
                print("ASSIST_END")

                with tracer.start_as_current_span("Server NF Items") as span:
                    span.set_attribute("llm.prompt", str(prompt.format_messages()))
                    span.set_attribute("llm.response", response_text)
                    span.set_attribute("llm.model", "ocigenai")
            except Exception as e:
                logger.error(f"Error: {e}", exc_info=True)
                print("Error:", e)

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        logger.info("Application interrupted")
    except Exception as e:
        logger.error(f"Application error: {e}", exc_info=True)

## Step 7: Create Gradio UI with Background Agent

In [ ]:
%%writefile gradio_app.py
import gradio as gr
import subprocess
import threading
import queue
import time
import os
import sys
import signal
from pathlib import Path

proc = None
stdout_queue = queue.Queue()
status_lock = threading.Lock()
agent_status = "Initializing"

def _reader_thread(stream, q):
    global agent_status
    for raw in iter(stream.readline, b""):
        try:
            line = raw.decode("utf-8", errors="replace").rstrip("\n\r")
        except:
            line = raw.decode(errors="replace").rstrip("\n\r")
        if "READY" in line:
            with status_lock:
                agent_status = "Active"
        q.put(line)
    with status_lock:
        agent_status = "Inactive"

def start_agent_process():
    global proc, agent_status
    if proc and proc.poll() is None:
        return
    with status_lock:
        agent_status = "Initializing"
    root = Path(os.getcwd())
    py = sys.executable
    main_path = str(root / "main.py")
    proc = subprocess.Popen(
        [py, "-u", main_path],
        cwd=str(root),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        stdin=subprocess.PIPE,
    )
    t = threading.Thread(target=_reader_thread, args=(proc.stdout, stdout_queue), daemon=True)
    t.start()

def stop_agent_process(timeout=3.0):
    global proc, agent_status
    if not proc:
        return
    if proc.poll() is None:
        try:
            proc.terminate()
            start = time.time()
            while proc.poll() is None and (time.time() - start) < timeout:
                time.sleep(0.1)
            if proc.poll() is None:
                proc.kill()
        except:
            try:
                proc.kill()
            except:
                pass
    proc = None
    with status_lock:
        agent_status = "Inactive"

start_agent_process()

def get_status():
    with status_lock:
        return agent_status

def send_message(message, history):
    if not message or not message.strip():
        return history, ""
    history = history + [{"role": "user", "content": message}]
    if get_status() != "Active":
        return history + [{"role": "assistant", "content": "Agent not ready. Status: " + get_status()}], ""
    try:
        if proc and proc.stdin:
            proc.stdin.write((message + "\n").encode("utf-8"))
            proc.stdin.flush()
        else:
            return history + [{"role": "assistant", "content": "Agent process not available"}], ""
    except Exception as e:
        return history + [{"role": "assistant", "content": f"Failed to send: {e}"}], ""

    response = None
    start = time.time()
    collecting_block = False
    block_lines = []
    while time.time() - start < 120.0:
        try:
            line = stdout_queue.get(timeout=0.25)
        except queue.Empty:
            continue
        if "READY" in line:
            with status_lock:
                agent_status = "Active"
            continue
        if line.strip() == "ASSIST_BEGIN":
            collecting_block = True
            block_lines = []
            continue
        if collecting_block:
            if line.strip() == "ASSIST_END":
                response = "\n".join(block_lines).strip()
                break
            else:
                block_lines.append(line)
            continue

    if response is None:
        response = "No response captured."

    return history + [{"role": "assistant", "content": response}], ""

CSS = """
.status-pill { display:inline-block; padding:4px 10px; border-radius:9999px; font-weight:600; font-size:0.9rem; color:#fff; }
.status-pill.active { background:#16a34a; }
.status-pill.initializing { background:#f59e0b; }
.status-pill.inactive { background:#6b7280; }
.spinner { width:10px; height:10px; border:2px solid #fff; border-right-color:transparent; border-radius:50%; display:inline-block; margin-right:8px; animation:spin 0.8s linear infinite; vertical-align:middle; }
@keyframes spin { to { transform: rotate(360deg); } }
"""

def _status_pill_html():
    s = get_status()
    cls = s.lower()
    if s == "Initializing":
        return f'<span class="status-pill {cls}"><span class="spinner"></span>{s}</span>'
    return f'<span class="status-pill {cls}">{s}</span>'

with gr.Blocks(title="OCI GenAI Bot", css=CSS) as demo:
    gr.Markdown("# OCI GenAI Bot - Invoice Resolution")
    with gr.Row():
        status_label = gr.HTML(_status_pill_html())
        refresh = gr.Button("Refresh Status", variant="secondary")
        restart = gr.Button("Restart Agent", variant="secondary")
    chat = gr.Chatbot(type="messages", height=480)
    msg = gr.Textbox(label="Message", placeholder="Type and press Enter...")
    clear = gr.Button("Clear")

    def _update_status():
        return _status_pill_html()

    refresh.click(_update_status, outputs=status_label)

    def _restart():
        stop_agent_process()
        start_agent_process()
        return _status_pill_html()

    restart.click(_restart, outputs=status_label)

    def _submit(m, h):
        new_hist, _ = send_message(m, h)
        return new_hist, "", _status_pill_html()

    msg.submit(_submit, inputs=[msg, chat], outputs=[chat, msg, status_label])
    clear.click(lambda: ([], ""), outputs=[chat, msg])
    demo.load(_update_status, outputs=status_label)

if __name__ == "__main__":
    demo.queue().launch(server_name="0.0.0.0", server_port=7860, share=False)

## Step 8: Set Up ngrok Tunnel

Get your ngrok auth token from: https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
from pyngrok import ngrok, conf
import getpass

# Prompt for ngrok auth token
ngrok_token = getpass.getpass("Enter your ngrok auth token: ")
conf.get_default().auth_token = ngrok_token

print("✅ ngrok configured successfully!")

## Step 9: Launch Application with ngrok

This will:
1. Start Phoenix observability on port 6006
2. Start the Gradio UI on port 7860
3. Create public ngrok URLs for both services

In [ ]:
import subprocess
import time
from pyngrok import ngrok

# Create ngrok tunnels
print("🌐 Creating ngrok tunnels...")

# Tunnel for Gradio UI (port 7860)
gradio_tunnel = ngrok.connect(7860, bind_tls=True)
gradio_url = gradio_tunnel.public_url

# Tunnel for Phoenix (port 6006)
phoenix_tunnel = ngrok.connect(6006, bind_tls=True)
phoenix_url = phoenix_tunnel.public_url

print("\n" + "="*60)
print("🎉 Application Started Successfully!")
print("="*60)
print(f"\n📱 Gradio UI (Public): {gradio_url}")
print(f"📊 Phoenix Dashboard (Public): {phoenix_url}")
print(f"\n🏠 Local URLs:")
print(f"   Gradio: http://localhost:7860")
print(f"   Phoenix: http://localhost:6006")
print("\n💡 Share these public URLs to access from anywhere!")
print("="*60)

# Launch Gradio app
print("\n🚀 Starting Gradio application...")
!python gradio_app.py

## Optional: Test the Setup

Run this cell to verify your configuration and database connection:

In [ ]:
# Quick configuration test
import os
from config_loader import load_properties, ensure_oci_config, apply_db_env
import oracledb

print("🔍 Testing Configuration...\n")

# Load config
PROPS = load_properties('config.properties')
ensure_oci_config(PROPS)
apply_db_env(PROPS)

print("✅ Configuration loaded")
print(f"   Model: {PROPS.get('OCI_GENAI_MODEL_ID')}")
print(f"   Embedding Model: {PROPS.get('OCI_GENAI_EMBEDDING_MODEL_ID')}")
print(f"   DB Alias: {PROPS.get('DB_ALIAS')}")

# Test DB connection
print("\n🔌 Testing Database Connection...")
try:
    WALLET_PATH = PROPS.get('WALLET_PATH', 'Wallet_ItsmyATPVectorDBs')
    DB_ALIAS = PROPS.get('DB_ALIAS')
    USERNAME = PROPS.get('USERNAME')
    PASSWORD = PROPS.get('PASSWORD')
    
    conn = oracledb.connect(
        user=USERNAME,
        password=PASSWORD,
        dsn=DB_ALIAS,
        config_dir=WALLET_PATH,
        wallet_location=WALLET_PATH,
        wallet_password=PASSWORD
    )
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM embeddings_products")
    count = cursor.fetchone()[0]
    cursor.close()
    conn.close()
    
    print(f"✅ Database connection successful")
    print(f"   Embeddings table has {count} records")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

print("\n✅ Setup verification complete!")

## Troubleshooting

**If the agent status stays "Initializing":**
- Check that all files were uploaded correctly
- Verify config.properties has correct credentials
- Check that embeddings_products table has data

**If database connection fails:**
- Ensure wallet files were extracted properly
- Verify DB_ALIAS matches the service name in tnsnames.ora
- Check credentials in config.properties

**If ngrok fails:**
- Verify you entered the correct auth token
- Check your ngrok account limits (free tier allows 1 agent)

**To view logs:**
- Main agent logs are printed to the notebook output
- Phoenix traces: visit the Phoenix public URL
- Gradio UI logs: check the cell output where gradio_app.py runs

## Example Queries

Once the application is running, try these queries in the Gradio UI:

1. **Complete invoice search:**
   ```
   Find invoice for Customer 43, product "Harry Potter", price 139.55, location RJ
   ```

2. **Product search only:**
   ```
   Search for products similar to "Harry Potter"
   ```

3. **Customer and location search:**
   ```
   Find invoices for Customer 43 in RJ state
   ```